In [1]:
!nvidia-smi

Thu May 24 00:11:31 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:29:00.0  On |                  N/A |
|  9%   51C    P8    19W / 280W |    314MiB / 11169MiB |     16%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import pip
for pkg in ['mxnet', 'mxnet-cu75', 'mxnet-cu80']:
    pip.main(['show',pkg])

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Name: mxnet-cu80
Version: 0.12.1
Summary: MXNet is an ultra-scalable deep learning framework. This version uses CUDA-8.0.
Home-page: https://github.com/dmlc/mxnet
Author: UNKNOWN
Author-email: UNKNOWN
License: Apache 2.0
Location: /home/dyjng/anaconda3/lib/python3.6/site-packages
Requires: numpy, graphviz, requests


You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import mxnet as mx
[mx.cpu(), mx.gpu(), mx.gpu(1)]

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


[cpu(0), gpu(0), gpu(1)]

In [4]:
from mxnet import nd
x = nd.array([1, 2, 3]) # (3, 1)
# x = nd.array([[1], [2], [3]])    # (3, 1)
# x = nd.array([[1, 2, 3]])   # (1, 3)
x.context
print(x.shape)

(3,)


In [5]:
a = nd.array([1, 2, 3], ctx = mx.gpu())
b = nd.zeros((3, 2), ctx = mx.gpu())
c = nd.random.uniform(shape = (2, 3), ctx = mx.gpu())
(a, b, c)

(
 [ 1.  2.  3.]
 <NDArray 3 @gpu(0)>, 
 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]
 <NDArray 3x2 @gpu(0)>, 
 [[ 0.32977498  0.43025011  0.70026755]
  [ 0.77781075  0.29912937  0.39169419]]
 <NDArray 2x3 @gpu(0)>)

In [6]:
import sys

try:
    nd.array([1, 2, 3], ctx = mx.gpu(5))
except mx.MXNetError as err:
    sys.stderr.write(str(err))

[00:11:33] src/storage/storage.cc:59: Check failed: e == cudaSuccess || e == cudaErrorCudartUnloading CUDA: invalid device ordinal

Stack trace returned 10 entries:
[bt] (0) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x26ab7c) [0x7faa64b0bb7c]
[bt] (1) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x261c4fd) [0x7faa66ebd4fd]
[bt] (2) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x261a024) [0x7faa66ebb024]
[bt] (3) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2077998) [0x7faa66918998]
[bt] (4) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(MXNDArrayCreateEx+0x145) [0x7faa6690ed55]
[bt] (5) /home/dyjng/anaconda3/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7faa9e637ec0]
[bt] (6) /home/dyjng/anaconda3/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call+0x22d) [0x7faa9e63787d]
[bt] (7) /home/dyjng/anaconda3/lib/python3.6/lib-dynload/_ctypes.cpytho

In [7]:
y = x.copyto(mx.gpu())  # x in CPU
z = x.as_in_context(mx.gpu())
(y, z, y is x, z is x)

(
 [ 1.  2.  3.]
 <NDArray 3 @gpu(0)>, 
 [ 1.  2.  3.]
 <NDArray 3 @gpu(0)>, False, False)

In [8]:
yy = y.as_in_context(mx.gpu())  # y in GPU
zz = z.copyto(mx.gpu())
# copyto always rebulit memory while as_in_context not if source and item all gpu/cpu
(yy is y, zz is z)

(True, False)

In [9]:
nd.exp(z + 2) * y


[  20.08553696  109.19629669  445.23950195]
<NDArray 3 @gpu(0)>

In [10]:
# x in cpu, y in gpu
try:
    x + y
except mx.MXNetError as err:
    sys.stderr.write(str(err))

[00:11:33] src/imperative/./imperative_utils.h:54: Check failed: inputs[i]->ctx().dev_mask() == ctx.dev_mask() (2 vs. 1) Operator broadcast_add require all inputs live on the same context. But the first argument is on cpu(0) while the 2-th argument is on gpu(0)

Stack trace returned 10 entries:
[bt] (0) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x26ab7c) [0x7faa64b0bb7c]
[bt] (1) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x214781c) [0x7faa669e881c]
[bt] (2) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2139e4e) [0x7faa669dae4e]
[bt] (3) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2090441) [0x7faa66931441]
[bt] (4) /home/dyjng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(MXImperativeInvokeEx+0x63) [0x7faa669317e3]
[bt] (5) /home/dyjng/anaconda3/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7faa9e637ec0]
[bt] (6) /home/dyjng/anaconda3/lib/python3.6/l

In [11]:
print(y)
# follow process will auto copy y to CPU primary memory
print(y.asnumpy())
print(y.sum().asscalar())


[ 1.  2.  3.]
<NDArray 3 @gpu(0)>
[ 1.  2.  3.]
6.0


In [12]:
from mxnet import gluon
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(1))

net.initialize(ctx = mx.gpu())

In [13]:
# data = nd.random_uniform(shape = [3, 2], ctx = mx.gpu())
net(y)


[[ 0.04118239]
 [ 0.08236478]
 [ 0.12354717]]
<NDArray 3x1 @gpu(0)>

In [14]:
net[0].weight.data()


[[ 0.04118239]]
<NDArray 1x1 @gpu(0)>